In [1]:
#Source for inspiration for the code below: Yağmur Cigdem Aktas on Medium
#https://towardsdatascience.com/huffman-encoding-python-implementation-8448c3654328

from typing import Union

class Node:
    def __init__(self, prob: Union[float, int], symbol: str, left = None, right = None):
        """
        Helper class to store values of each symbol in source ensemble.
        Parameters:
        ---------------------
        prob : float
            Probability of the symbol
        symbol : str
            The symbol itself in a string form
        left : Node
            The left child of node. Only used when constructing the tree. Default None.
        right : Node
            The left child of node. Only used when constructing the tree. Default None.
        """
        self.prob = prob
        self.symbol = symbol
        self.left = left
        self.right = right
        self.code = ''

def Calculate_Codes(node, codes: dict[str, str] = None, val: str = None) -> dict[str, str]:
    """
    Helper function to calculate the codes from a built tree.
    Parameters:
    -----------------
    node : Node
        Node from which to calculate codes. Initial call should be a root node of the built tree.
    codes : Dictionary[str, str]
        Dictionary of symbols and their associated codes, both in strings. Default = dict()
    val : str

    Return:
    Dict[str, str] : Dictionary of symbols and their associated codes, both in strings.
    """
    #better to put mutable objects in code rather than in argument
    if not codes:
        codes = dict()
    if not val:
        val = ''
    
    #calculate the new code for the current node
    new_val = val + str(node.code)

    #go down the left branch
    if node.left:
        codes = Calculate_Codes(node.left, codes, new_val)

    #go down the right branch
    if node.right:
        codes = Calculate_Codes(node.right, codes, new_val)

    #only store code if its a leaf node
    if (not node.left and not node.right):
        codes[node.symbol] = new_val

    return codes


def build_huff_tree(source_symb: list[str], prob_lst: list[Union[float, int]]):
    """
    Calculate the huffman codes for a given ensemble
    Parameters
    ------------
    source_symb : List of source symbols, should be a list of strings
    prob_lst : Probability distribution list for source symbols in the given order

    Return
    ------------
    Node : Root node of tree built for huffman codes
    """
    nodes = []

    #create all the nodes
    for i in range(len(source_symb)):
        nodes.append(Node(prob_lst[i], source_symb[i]))
    
    #construct the tree
    while len(nodes) > 1:
        #sort the nodes, could also create a heap instead
        nodes = sorted(nodes, key = lambda x: x.prob)

        #pick the smallest nodes
        right = nodes[0]
        left = nodes [1]

        #assign a bit to each of the nodes
        left.code = 0
        right.code = 1

        #combine the smallest nodes
        newNode = Node(left.prob + right.prob, left.symbol + right.symbol, left, right)

        #remove the smallest nodes, add the new node and run again
        nodes.remove(left)
        nodes.remove(right)
        nodes.append(newNode)

    return nodes[0]

def encode_input(huff_codes: dict[str, str], input_seq: str) -> str:
    """
    Use Huffman codes to encode the input sequence.
    Parameters
    ----------
    huff_codes : dict[str, str]
        Dictionary of symbols and their codes
    input_seq : str
        Sequence to encode
    Return
    --------
    str : Encoded string
    """
    #get the size of each symbol
    sym_size = len(list(huff_codes)[0])
    #length to iterate over
    iter_len = len(input_seq)-(sym_size-1)
    
    encoded_output = ''.join([str(huff_codes[input_seq[c:c+sym_size]]) for c in range(0, iter_len, sym_size)])
    return encoded_output

def decode_input(root_node, input_seq: str) -> str:
    """
    Use Huffman codes to encode the input sequence.
    Parameters
    ----------
    root_node : Node
        Root node for the huffman tree
    input_seq : str
        Sequence to decode
    
    Return
    --------
    str : Decoded string
    """
    node = root_node            #start the search from root node
    decoded_output = ''         #initialize the decoded output
    for bit in input_seq:
        #go down the right of the tree for value 1
        if bit == '1':
            node = node.right
        #go down the left of the tree for value 0
        else:
            assert(bit == '0')
            node = node.left
        
        #get the symbol and decode if reached leaf node
        if (not node.left and not node.right):
            decoded_output += node.symbol
            node = root_node    #restart at the top of the tree
            
    return decoded_output

In [2]:
#Pre-class work
source_symb = ['0', '1', '2']
prob_lst = [0.2, 0.4, 0.4]
input_seq = "210"

root_node = build_huff_tree(source_symb, prob_lst)
huff_codes = Calculate_Codes(root_node)
output = encode_input(huff_codes, input_seq)
print(huff_codes)
print(output)
decoded = decode_input(root_node, output)
print(decoded)

{'1': '00', '0': '01', '2': '1'}
10001
210


In [3]:
import requests
import random
import collections
import numpy as np
import scipy.stats as sts

#download Shakespeare
data = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt').content.decode('utf-8')
print('Text length:', len(data))
print('\nFirst 200 characters:', data[:200])
i = random.randint(0, len(data) - 201)
print('\nRandom 200 characters:', data[i:i+200])

# Count pairs of characters
symbol_counts = collections.defaultdict(int)

#not sure why you started the range with 1, but since the results are the same,
#I'll keep it for consistency
for i in range(1, len(data)):
    symbol_counts[data[i:i+2]] += 1
symbols = list(symbol_counts.keys())
counts = list(symbol_counts.values())
print(f'\nThere are {len(symbols)} character pairs')
print('Entropy of the source:', sts.entropy(counts, base=2))

# Sort the character pairs from lowest count to highest count
index = np.argsort(counts)
symbols = list(np.array(symbols)[index])
counts = list(np.array(counts)[index])
print('\n10 most common character pairs:', ', '.join(repr(_) for _ in symbols[-10:][::-1]))


Text length: 5458199

First 200 characters: This is the 100th Etext file presented by Project Gutenberg, and
is presented in cooperation with World Library, Inc., from their
Library of the Future and Shakespeare CDROMS.  Project Gutenberg
often

Random 200 characters:    Farewell, my blood; which if to-day thou shed,
    Lament we may, but not revenge thee dead.
  BOLINGBROKE. O, let no noble eye profane a tear
    For me, if I be gor'd with Mowbray's spear.
    As

There are 2259 character pairs
Entropy of the source: 8.16815712274428

10 most common character pairs: '  ', 'e ', '\n ', ' t', 'th', 'he', 't ', 's ', ' a', ', '


In [4]:
root_node_shakespeare = build_huff_tree(symbols, counts)
huff_codes_shakespeare_2 = Calculate_Codes(root_node_shakespeare)

expected_length = 0
for key in huff_codes_shakespeare_2.keys():
    expected_length += len(huff_codes_shakespeare_2[key])*symbol_counts[key]

print('Entropy of the source:', sts.entropy(counts, base=2))
print('Expected length of the code:', expected_length/sum(counts))

Entropy of the source: 8.16815712274428
Expected length of the code: 8.197100031915294


In [5]:
input_seq_shakespeare = "The cat sat on the mat, and ate a fat rat."
print(f"Encoding the following sequence: {input_seq_shakespeare}")

coded_seq_shakespeare = encode_input(huff_codes_shakespeare_2, input_seq_shakespeare)
print(f"Encoded sequence: {coded_seq_shakespeare}")
print(f"Length of the compressed string in bits is {len(coded_seq_shakespeare)}.")

print(f"Decoding the output to perform the check.....")
decoded_seq_shakespeare = decode_input(root_node_shakespeare, coded_seq_shakespeare)
print(f"Decoded sequence: {decoded_seq_shakespeare}")

Encoding the following sequence: The cat sat on the mat, and ate a fat rat.
Encoded sequence: 0001010000000010001001100101100001101111010110000001111001100100111011000010111101110100101100111101101111000000100101011100101110001011000011110110110100001
Length of the compressed string in bits is 157.
Decoding the output to perform the check.....
Decoded sequence: The cat sat on the mat, and ate a fat rat.
